In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Pour le calcul de similarité
from sklearn.metrics.pairwise import cosine_similarity

# Pour la modélisation supervisée
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, f1_score, classification_report



In [16]:
# Définition des chemins de fichiers
movies_path = "C:/Users/Victor/Documents/A_Projet_MLOP/Template_MLOps_movie_recommandation-master/data/processed/movie_matrix.csv"
users_path = "C:/Users/Victor/Documents/A_Projet_MLOP/Template_MLOps_movie_recommandation-master/data/processed/user_matrix.csv"

# Vérification et chargement des fichiers
if os.path.exists(movies_path) and os.path.exists(users_path):
    movies_df = pd.read_csv(movies_path)
    users_df = pd.read_csv(users_path)
    
    # Affichage d'un aperçu
    print("Movies Matrix:")
    print(movies_df.head())
    print("\nUser Matrix:")
    print(users_df.head())
else:
    print("❌ Erreur : Un ou plusieurs fichiers sont introuvables.")
    if not os.path.exists(movies_path):
        print(f"Fichier manquant : {movies_path}")
    if not os.path.exists(users_path):
        print(f"Fichier manquant : {users_path}")



Movies Matrix:
   movieId  (no genres listed)  Action  Adventure  Animation  Children  \
0        1                   0       0          1          1         1   
1        2                   0       0          1          0         1   
2        3                   0       0          0          0         0   
3        4                   0       0          0          0         0   
4        5                   0       0          0          0         0   

   Comedy  Crime  Documentary  Drama  ...  Film-Noir  Horror  IMAX  Musical  \
0       1      0            0      0  ...          0       0     0        0   
1       0      0            0      0  ...          0       0     0        0   
2       1      0            0      0  ...          0       0     0        0   
3       1      0            0      1  ...          0       0     0        0   
4       1      0            0      0  ...          0       0     0        0   

   Mystery  Romance  Sci-Fi  Thriller  War  Western  
0        0 

In [17]:
# Vérifier les valeurs manquantes
print("Valeurs manquantes dans movies_df :", movies_df.isnull().sum().sum())
print("Valeurs manquantes dans users_df :", users_df.isnull().sum().sum())

# Supprimer la colonne '(no genres listed)' si elle existe
if '(no genres listed)' in movies_df.columns:
    movies_df.drop(columns=['(no genres listed)'], inplace=True)
if '(no genres listed)' in users_df.columns:
    users_df.drop(columns=['(no genres listed)'], inplace=True)

# Afficher les doublons éventuels
print("Doublons dans movies_df :", movies_df.duplicated().sum())
print("Doublons dans users_df :", users_df.duplicated().sum())


Valeurs manquantes dans movies_df : 0
Valeurs manquantes dans users_df : 0
Doublons dans movies_df : 0
Doublons dans users_df : 0


In [18]:
# Extraire le vecteur de l'utilisateur 1 (en excluant la colonne userId)
user_id = 1
user_vector = users_df[users_df['userId'] == user_id].iloc[:, 1:].values.flatten()

# Extraire la matrice des films (en excluant la colonne movieId)
movie_features = movies_df.iloc[:, 1:].values

# Calcul de la similarité cosine entre l'utilisateur et chaque film
similarities = cosine_similarity(movie_features, user_vector.reshape(1, -1)).flatten()

# Ajouter la similarité au dataframe des films
movies_df['similarity'] = similarities

# Afficher les similarités calculées pour vérification
print(movies_df[['movieId', 'similarity']].head())


   movieId  similarity
0        1    0.384306
1        2    0.194745
2        3    0.539494
3        4    0.839259
4        5    0.481869


In [19]:
# Définir un seuil (par exemple, la médiane)
threshold = np.median(similarities)
print("Seuil utilisé :", threshold)

# Créer le label : 1 si similarity >= seuil, sinon 0
movies_df['label'] = (movies_df['similarity'] >= threshold).astype(int)

# Vérifier la répartition des labels
print(movies_df['label'].value_counts())


Seuil utilisé : 0.5281363636181959
label
1    13674
0    13604
Name: count, dtype: int64


In [20]:
# On sélectionne les features (colonnes de genres) et la cible
# On suppose que la première colonne est 'movieId', ensuite les genres se trouvent jusqu'à la colonne 'similarity'
features = movies_df.columns[1:-2]  # Excluant 'movieId', 'similarity' et 'label'
X = movies_df[features]
y = movies_df['label']

print("Dimensions de X :", X.shape)
print("Dimensions de y :", y.shape)


Dimensions de X : (27278, 19)
Dimensions de y : (27278,)


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print("Entraînement :", X_train.shape, "Test :", X_test.shape)

Entraînement : (21822, 19) Test : (5456, 19)


In [22]:
# Instanciation du modèle de régression logistique
clf = LogisticRegression(max_iter=1000, random_state=42)

# Entraînement du modèle
clf.fit(X_train, y_train)

# Prédictions sur l'ensemble de test
y_pred = clf.predict(X_test)


In [23]:
# Calcul et affichage des métriques
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy :", accuracy)
print("Recall :", recall)
print("F1-score :", f1)

# Rapport détaillé
print("\nClassification Report :\n", classification_report(y_test, y_pred))


Accuracy : 0.9961510263929618
Recall : 0.9970749542961609
F1-score : 0.9961643835616438

Classification Report :
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2721
           1       1.00      1.00      1.00      2735

    accuracy                           1.00      5456
   macro avg       1.00      1.00      1.00      5456
weighted avg       1.00      1.00      1.00      5456



In [24]:
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100]
}

grid_search = GridSearchCV(LogisticRegression(max_iter=1000, random_state=42), param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Meilleur paramètre :", grid_search.best_params_)
print("Meilleur F1-score en validation :", grid_search.best_score_)

# Évaluation sur le test avec le meilleur modèle
best_clf = grid_search.best_estimator_
y_pred_best = best_clf.predict(X_test)
print("\nAprès tuning - Classification Report :\n", classification_report(y_test, y_pred_best))


Meilleur paramètre : {'C': 100}
Meilleur F1-score en validation : 0.9987199578409799

Après tuning - Classification Report :
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2721
           1       1.00      1.00      1.00      2735

    accuracy                           1.00      5456
   macro avg       1.00      1.00      1.00      5456
weighted avg       1.00      1.00      1.00      5456

